# Module: Fetch Data
*get data from TSETMC and directly write to db*

## Prepearing Classification Data

In [2]:
import pandas as pd
import requests
import zeep
import json
import pytse_client

from utils import logger_app, ar_to_fa_series, ar_to_fa

In [3]:
pd.set_option('display.max_columns', 30)

## Definition & Assignment

In [4]:
request_headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Referer': 'http://main.tsetmc.com/StaticContent/WebServiceHelp'
}
cookie_jar = {'ASP.NET_SessionId': 'wa40en1alwxzjnqehjntrv5j'}

IDENTITY_URL = "http://cdn.tsetmc.com/api/Instrument/GetInstrumentIdentity/{}"
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']

In [5]:
def last_possible_deven():
    client = zeep.Client(
        'http://service.tsetmc.com/WebService/TseClient.asmx?wsdl')
    last_workday = client.service.LastPossibleDeven()
    return last_workday.split(';')[0] if last_workday.split(
        ';')[1] == last_workday.split(';')[0] else print(
            last_workday)  # for debuging purposes

In [6]:
def InstrumentAndShare(last_fetch_date, last_record_id):
    client = zeep.Client(
        wsdl='http://service.tsetmc.com/WebService/TseClient.asmx?wsdl')

    instruments, share_increase = client.service.InstrumentAndShare(
        last_fetch_date, last_record_id).split('@')

    if instruments:
        instruments = instruments.split(';')
        instruments = pd.DataFrame(
            [instrument.split(',') for instrument in instruments])
        instruments.columns = [
            'id', 'inst_code_12', 'inst_code_5', 'name_latin',
            'company_code_4', 'ticker', 'name', 'isin', 'mod_date',
            'market_code', 'company_name', 'status_code', 'group_code',
            'market_type_code', 'tableu_code', 'industry_sector_code',
            'industry_subsector_code', 'type_code'
        ]
        instruments.set_index('id', inplace=True)
        instruments['ticker'] = ar_to_fa_series(instruments['ticker'])
        instruments['name'] = ar_to_fa_series(instruments['name'])
        instruments['company_name'] = ar_to_fa_series(
            instruments['company_name'])

    if share_increase:
        share_increase = share_increase.split(';')
        share_increase = pd.DataFrame(
            [share.split(',') for share in share_increase])
        share_increase.columns = [
            'record_id', 'id', 'date', 'before_raise', 'after_raise'
        ]
        #capital_increase['record_id'] = capital_increase['record_id'].astype(int)
        share_increase.set_index('record_id', inplace=True)
    else:
        share_increase = None

    return instruments, share_increase


In [7]:
def flatten_json(y):
    out = {}
    
    def flatten(x, name =''):
        
        # If the Nested key-value 
        # pair is of dict type
        if type(x) is dict:
            
            for a in x:
                flatten(x[a], name + a + '_')
                
        # If the Nested key-value
        # pair is of list type
        elif type(x) is list:
            
            i = 0
            
            for a in x:                
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
        
    flatten(y)
    return out

In [8]:
def get_identity(id):
    result = requests.get(
        IDENTITY_URL.format(id),
        headers=request_headers,
        cookies=cookie_jar)
    result = json.loads(result.text)
    return flatten_json(result['instrumentIdentity'])

In [9]:
last_date = last_possible_deven()
insts, shares = InstrumentAndShare(0,0)
print(last_possible_deven())

20230903


In [10]:
all_identities = pd.DataFrame({index:get_identity(index) for index in insts[insts['tableu_code'] != '6'].index}).transpose()

In [15]:
all_identities.to_csv('all_idz.csv')

## Categorization